In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [ ]:
df.reset_index(inplace= True)

### **P4:** Spielt die Vergangenheit eines Users eine Rolle in Hinblick auf die Trinkgeldwahrscheinlichkeit, oder kommt es nur auf Inhalt und Parameter der aktuellen Bestellung an?

#### User-Vergangenheit

In [ ]:
# aktuelle Bestellung raussnehmen und tip_wshk berechnen und mit aktueller Bestellung vergleichen

In [129]:
df_userhh = df.copy()

In [136]:
df_userh1 = df_userhh.groupby("user_id", as_index= False).agg(orders = ("order_id", "unique"), reordered= ("reordered", "mean"), tip= ("tip", "mean"))

In [132]:
last_orders = []

for ind, order in enumerate(df_userh1.orders):
 last_orders.append(df_userh1.orders[ind][-1])

In [135]:
df_userhh.drop(df_userhh.loc[df_userhh.order_id.isin(last_orders)].index, inplace= True)

In [137]:
df_userh1.orders[0][-1]

560872

In [140]:
df_userh = df_userhh.groupby("user_id", as_index= False).agg(orders = ("order_id", "nunique"), reordered= ("reordered", "mean"), tip= ("tip", "mean"))

In [141]:
df_userh

,user_id,orders,reordered,tip
0,3,11,0.612500,0.925000
1,7,20,0.668293,0.668293
2,14,13,0.325359,0.478469
3,23,4,0.100000,0.000000
4,25,2,0.058824,0.352941
...,...,...,...,...
37362,206186,3,0.241379,0.758621
37363,206189,5,0.542857,0.142857
37364,206192,13,0.644444,0.255556
37365,206195,19,0.432836,0.149254


In [147]:
df_last_order = df.loc[df.order_id.isin(last_orders)].groupby("user_id").agg(tip_last_order= ("tip", "max")).reset_index()

In [153]:
df_userh = df_userh.merge(df_last_order, how= "right", on= "user_id")

**Bestellungen bei welcher der user eine tip_wshk von >= 0.5 hat verglichen mit seiner letzten/aktuellen Bestellung**


Unter der Annahme, dass wenn die tip_wshk eines users in der historie >= 0.5 ist, tendiert er eher dazu zu tippen bei seiner letzten/aktuellen Bestellung

**trifft nicht zu:**

In [154]:
df_userh.loc[(df_userh.tip >= 0.5) & (df_userh.tip_last_order == 0)] #! Hier schauen ob die Attribute die Bestellung beeinflussen

,user_id,orders,reordered,tip,tip_last_order
0,3,11,0.612500,0.925000,0
1,7,20,0.668293,0.668293,0
6,39,6,0.588710,0.685484,0
22,118,27,0.689441,0.763975,0
33,169,6,0.123596,0.876404,0
...,...,...,...,...,...
37322,205967,10,0.320755,0.547170,0
37351,206140,9,0.473282,0.664122,0
37354,206153,13,0.506849,0.636986,0
37358,206162,18,0.457447,0.537234,0


**trifft zu:**

In [155]:
df_userh.loc[(df_userh.tip >= 0.5) & (df_userh.tip_last_order == 1)]

,user_id,orders,reordered,tip,tip_last_order
13,73,6,0.363636,0.636364,1
15,85,6,0.482759,0.827586,1
20,110,20,0.569405,0.643059,1
21,111,7,0.107143,1.000000,1
24,128,11,0.242105,1.000000,1
...,...,...,...,...,...
37334,206038,7,0.375000,1.000000,1
37342,206086,46,0.832477,0.580678,1
37344,206093,17,0.605263,0.728070,1
37348,206130,16,0.735294,0.852941,1


#### old

In [ ]:
df_userh.loc[df_userh.orders < df_userh.orders]

In [ ]:
df_userh.loc[df_userh.orders >= df_userh.orders.quantile(0.8)]

In [ ]:
list_bins = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
df_userh["orders_cut"] = pd.cut(df_userh["orders"], bins= list_bins, include_lowest= True)

In [ ]:
df_userh

In [ ]:
df_userh1 = df_userh.groupby("orders_cut", as_index= False).agg(orders= ("orders", "sum"), reordered= ("reordered", "mean"), tip= ("tip", "mean"))
df_userh1

In [ ]:
df_userh1["orders_cut"] = df_userh1["orders_cut"].astype("str")

In [ ]:
px.bar(df_userh1, x= "orders_cut", y= "tip")

#### Inhalt und Parameter einer Bestellung

In [ ]:
# wieder aktuelle bestellung rausnehmen und tipp_wshk frühere mit aktueller vgl und warenkorb vergeleichen sowie parameter county, department